<a href="https://colab.research.google.com/github/zhuhuachi/zhuhuachi/blob/main/5_1_%E4%BD%BF%E7%94%A8_Keras_%E6%9E%84%E5%BB%BA%E5%BC%BA%E5%A4%A7%E7%9A%84%E5%9B%BE%E5%83%8F%E5%88%86%E7%B1%BB%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C_%E7%8C%AB%E7%8B%97%E7%9A%84%E4%BE%8B%E5%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. 使用 Keras 构建强大的图像分类卷积神经网络 - 猫狗的例子

我们将使用[数据集](https://www.kaggle.com/chetankv/dogs-cats-images)将图像分类为猫或狗。

我已将图像存储在一个目录结构中，如下所示

![8NSOmi](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/8NSOmi.jpg)

构建 CNN 以对图像进行分类的高级步骤是

* 通过应用内核或特征图创建卷积层
* 为平移不变性应用 Max pool
* 展平输入
* 创建一个全连接的神经网络
* 训练模型
* 预测输出

![EM3n1y](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/EM3n1y.jpg)

首先，我们初始化用于构建 CNN 的神经网络

In [ ]:
from keras.models import Sequential
classifier = Sequential()

我们在输入图像上使用多个特征检测器或内核应用卷积(convolution)操作。特征检测器(multiple feature detector)可以用于锐化(sharpen)图像、模糊(blur)图像等。

我们的输入图像是一个 64 x 64 像素的彩色图像，具有 3 个通道。

我们想要 32 个特征图(feature map)，通过使用 3 x 3 内核(kernel)或特征检测器，从左到右(from left to right)的步长(stride)为 1，从上到下(from top to bottom)的步长为 1。
激活函数是 relu - 整流器线性单元，它有助于神经网络中的非线性。

In [ ]:
from keras.layers import Conv2D
classifier.add(Conv2D(filters=32, kernel_size=(3,3),strides=(1, 1), input_shape=(64,64,3), activation='relu'))

一旦我们有了 32 个特征图（feature maps），我们就会应用最大池化（max pooling）来实现平移不变性（translational invariance）。平移不变性是当我们改变少量输入时，输出不会改变。最大池化减少了单元格的数量（Max pooling reduces the number of cells）。

池化有助于检测颜色、边缘等特征。
对于最大池化，我们对所有 32 个特征图使用 2 x 2 矩阵的 pool_size。

In [ ]:
from keras.layers import MaxPooling2D
classifier.add(MaxPooling2D(pool_size=(2,2)))

我们可以再添加一个卷积层。

这次我们将有 64 个特征图(feature maps)，其内核为 (3,3)。默认步幅为 (1,1)。然后我们将最大池化应用于卷积层(convolutional layers.)。

In [ ]:
classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

下一步是展平(flatten)所有输入。展平的数据将作为全连接神经网络(fully connected neural network)的输入。

In [ ]:
from keras.layers import Flatten
classifier.add(Flatten())

我们现在构建一个具有 128 个输入单元(input units )和一个输出单元(one output unit)的全连接神经网络。我们使用 20% 的 Dropout 率来防止(prevent)过拟合(overfitting)。

这是一个二元分类问题，因此我们在输出层使用 sigmoid 激活函数。

In [ ]:
from keras.layers import Dense
from keras.layers import Dropout
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(rate=0.2))
classifier.add(Dense(units=1, activation='sigmoid'))

我们现在用 **Adadelta** 优化器编译神经网络。 **Adadelta** 加速了收敛(convergence)。

损失函数将是 **binary_crossentropy**，因为这是一个二元分类问题。

In [ ]:
classifier.compile( optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

我们通过许多随机变换(a number of random transformations)应用图像增强( image augmentation)，使 CNN 适合图像。

我们缩放(zoom)图像，剪切(shear)图像并水平翻转(horizontally flip )图像。这有助于防止过度拟合(overfitting)并帮助模型更好地泛化(generalize better)。

我们的原始图像包含 0-255 范围内的 RGB 系数。在给定典型学习率的情况下，这些值对于我们的模型来说太高了，无法处理。为了解决这个问题，我们通过使用 1/255 因子进行缩放来将目标值设置在 0 和 1 之间。

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# applying transformation to image
train_datagen = ImageDataGenerator(
 rescale=1./255,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

我们创建训练集(training)和测试集(test set)。我们的目标尺寸应该与输入图像的输入尺寸（64, 64）相匹配。

由于我们的数据存储在目录中，我们使用 **flow_from_directory** 方法。 flow_from_directory 从指定路径获取数据帧并生成批量增强的规范化(augmented normalized)数据。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
training_set = train_datagen.flow_from_directory(
        'drive/MyDrive/Colab Notebooks/dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        'drive/MyDrive/Colab Notebooks/dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8020 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


我们最终使用 **fit_genator** 将数据拟合到我们上面创建的 CNN 模型中。

我们使用 fit_genator 时

* 数据集通常太大而无法放入内存。
* 当我们需要执行数据增强以避免过度拟合时。这增加了我们模型的泛化能力。

要设置参数的值，我们可以使用以下公式，但这不是硬性规定。


steps_per_epoch = 总训练样本(Total Training Samples)/训练批次大小 (Training Batch Size)
validation_steps = 总验证样本(Total Validation Samples)/验证批次大小(Validation Batch Size)

我们在训练数据中有 8000 张图像，我们的训练批量大小为 32，因此 steps_per_epoch 设置为 8000/32。

我们在测试集中有 2000 张图像，我们的批量大小为 32，因此 validation_steps = 2000/32 四舍五入为 64。我没有得到很好的验证准确度，所以我尝试了不同的组合，然后确定为 150

In [ ]:
from IPython.display import display 

classifier.fit_generator(
        training_set,
        steps_per_epoch=250,
        epochs=10,
        validation_data=test_set, validation_steps=150)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10
250/250 [==============================] - 1445s 6s/step - loss: 0.6953 - accuracy: 0.4926 - val_loss: 0.6939 - val_accuracy: 0.5070
Epoch 2/10
250/250 [==============================] - 46s 183ms/step - loss: 0.6942 - accuracy: 0.5008
Epoch 3/10
250/250 [==============================] - 46s 183ms/step - loss: 0.6934 - accuracy: 0.5084
Epoch 4/10
250/250 [==============================] - 46s 184ms/step - loss: 0.6925 - accuracy: 0.5205
Epoch 5/10
250/250 [==============================] - 47s 188ms/step - loss: 0.6912 - accuracy: 0.5317
Epoch 6/10
250/250 [==============================] - 46s 185ms/step - loss: 0.6903 - accuracy: 0.5378
Epoch 7/10
250/250 [==============================] - 47s 186ms/step - loss: 0.6899 - accuracy: 0.5384
Epoch 8/10
250/250 [==============================] - 46s 185ms/step - loss: 0.6896 - accuracy: 0.5414
Epoch 9/10
250/250 [==============================] - 46s 184ms/step - loss: 0.6876 - accuracy: 0.5576
Epoch 10/10
250/250 [===========

![Tyespx](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/Tyespx.jpg)

我们在训练数据上获得了 99% 的准确率，在测试数据上获得了 77% 的准确率。

我们现在终于拍摄了一张图像来进行预测。我已将狗和猫的测试图像添加到名为 single_prediction 的数据集下的新文件夹中。

![ki00V4](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/ki00V4.jpg)

我们如何确定 0 是代表猫还是狗？

我们使用 class_indices 作为 training_set 来理解 0 和 1 代表什么。

In [ ]:
training_set.class_indices

![XuhIPt](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/XuhIPt.jpg)

如果输出为 0，则图像是猫，如果输出为 1，则图像是狗。

我们如何对图像进行预测？

我们需要对测试图像输入进行整形，以使预测方法正常工作。

**test_image** 是一个 **64 x 64** 像素的输入。我们首先需要为颜色添加 3 个通道，以匹配我们为第一个卷积层指定的输入形状。为此，我们使用 Keras 的图像库。将 img_to_array() 方法应用于 test_image 后，它的维度为 (64, 64, 3)

**Predict** 方法还需要一个 **batch_size**，它是输入图像的第一个维度。

Batch_size 指定我们将发送到 predict 方法的图像数量。在我们的示例中，我们只发送一张图像，但我们仍然需要指定它。

test_image 的最终尺寸为 (1, 64, 64, 3)

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img("D:\\ML-data\\dataset\\single_prediction\\cat_or_dog_1.jpg",target_size=(64, 64) )
# Adding the channel
test_image = image.img_to_array(test_image)
# adding the batch size as predict method expects
test_image = np.expand_dims(test_image, axis=0)
# Predicting the test image
result= classifier.predict(test_image)
print(result)

我们得到的结果是 1，这意味着狗被正确分类的第一张图像。

我们可以通过添加更多的卷积层或增加全连接层的深度来进一步微调模型。